In [1]:
# python -m rule_engine.debug_repl --edit-console
import rule_engine

In [2]:
from enum import IntEnum

In [3]:
class ObsState(IntEnum):
    EMPTY = 0
    IDLE = 1

In [4]:
class DeviceObsState(object):
    def __init__(self, csp_subarray, sdp_subarray, obs_state):
        self.csp_subarray = csp_subarray
        self.sdp_subarray = sdp_subarray
        self.obs_state = obs_state

In [9]:
empty_state_rule = rule_engine.Rule(f"csp_subarray_obsState == {int(ObsState.EMPTY)} and sdp_subarray_obsState == {int(ObsState.EMPTY)}")

In [14]:
devices = [DeviceObsState("csp/001", "sdp/001", ObsState.IDLE), DeviceObsState("csp/002", "sdp/002", ObsState.EMPTY)]

In [15]:
devices

In [16]:
rule_engine_match_dict = {"csp_subarray_obsState": int(devices[0].obs_state), "sdp_subarray_obsState": int(devices[1].obs_state)}

In [17]:
empty_state_rule.matches(rule_engine_match_dict)

False

In [32]:
context = rule_engine.Context(type_resolver=rule_engine.type_resolver_from_dict({
    'obsState': rule_engine.DataType.STRING,
    'pointingState': rule_engine.DataType.SET
}))

In [39]:
obs_state_rule = rule_engine.Rule("obsState == '0' and pointingState == {1}", context=context)

In [40]:
obs_state_rule.matches({"obsState": "0", "pointingState": set([1])})

True

In [41]:
rule_file = open("rules.rule", "r")
rules = rule_file.read()
rule_file.close()

In [42]:
rules

"obsState == '1' and pointingState == {2, 3}"

In [43]:
idle_rule = rule_engine.Rule(rules, context=context)

In [44]:
idle_rule

<Rule text="obsState == '1' and pointingState == {2, 3}" >

In [49]:
idle_rule.matches({"obsState": "01", "pointingState": set([2, 3])})

False

In [53]:
filter_rules = idle_rule.filter([{"obsState": "1", "pointingState": set([2, 3]), "device": "csp"}, {"obsState": "1", "pointingState": set([2, 3]), "device": "sdp"}, {"obsState": "02", "pointingState": set([2, 3]), "device": "dish"}])

In [54]:
list(filter_rules)

[{'obsState': '1', 'pointingState': {2, 3}, 'device': 'csp'},
 {'obsState': '1', 'pointingState': {2, 3}, 'device': 'sdp'}]

In [120]:
with open("rules.rule", "r") as f:
    rule_list = f.readlines()

In [121]:
rule_list

["obsState == '1' and pointingState.to_ary == [2, 3]\n",
 "name[:3] == 'alc' and writer == 'paul'\n",
 'name == "alchemist" ? true : writer == "paul" ? true : false\n',
 'name == "alc" or name == "krishna"\n',
 'devices.length == 3\n',
 'state.to_int == 1 \n']

In [122]:
book_rule = rule_engine.Rule(rule_list[1].replace("\n", ""))

In [123]:
book_rule.matches({"name": "alchemist", "writer": 'paul'})

True

In [124]:
book_name_rule = rule_engine.Rule(rule_list[2].replace("\n", ""))

In [125]:
book_name_rule.matches({"name": "alchemist", "writer": 'pa'})

True

In [126]:
name_or_rule = rule_engine.Rule(rule_list[3].replace("\n", ""))

In [127]:
name_or_rule.matches({"name": "krishna", "writer": "ashwin"})

True

In [128]:
idle_rule = rule_engine.Rule(rule_list[0].replace("\n", ""))

In [129]:
idle_rule.matches({"obsState": "1", "pointingState": {2, 3}, "device": "csp"})

True

In [130]:
device_rule = rule_engine.Rule(rule_list[4].replace("\n", ""))

In [131]:
device_rule.matches({"devices": ["csp", "sdp"]})

False

In [132]:
device_rule.matches({"devices": ["csp", "sdp", "dish"]})

True

In [133]:
state_rule = rule_engine.Rule(rule_list[5].replace("\n", ""))

In [137]:
state_rule.matches({"state": "1"})

True

In [138]:
state_rule.matches({"state": "2"})

False